In [ ]:
import pandas as pd
import numpy as np

In [2]:
pbp = pd.read_csv('../../Data/play-by-play/pbp_exp9.csv')

In [3]:
pbp

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,OffenseTeam,DefenseTeam,side_of_field,...,home_team_on_defense,offense_formation,defenders_in_box,pass_count,run_count,Previous_DEF_rank,current_defence_rank,Player-Injured-On-Play,Temperature,Precipitation
0,35,2019_01_GB_CHI,2019090500,CHI,GB,REG,1,GB,CHI,CHI,...,1,KICKOFF,0.0,0,0,1,12,0,65.630000,0.0
1,50,2019_01_GB_CHI,2019090500,CHI,GB,REG,1,GB,CHI,GB,...,2,I_FORM,7.0,0,1,1,12,0,65.630000,0.0
2,71,2019_01_GB_CHI,2019090500,CHI,GB,REG,1,GB,CHI,GB,...,3,I_FORM,8.0,1,1,1,12,0,65.630000,0.0
3,95,2019_01_GB_CHI,2019090500,CHI,GB,REG,1,GB,CHI,GB,...,4,SHOTGUN,5.0,2,1,1,12,0,65.630000,0.0
4,125,2019_01_GB_CHI,2019090500,CHI,GB,REG,1,GB,CHI,GB,...,5,PUNT,0.0,2,1,1,12,0,65.630000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88331,4256,2020_21_KC_TB,2021020700,TB,KC,POST,21,KC,TB,TB,...,87,SHOTGUN,5.0,81,46,17,5,0,59.848571,0.0
88332,4280,2020_21_KC_TB,2021020700,TB,KC,POST,21,KC,TB,TB,...,88,SHOTGUN,6.0,82,46,17,5,0,59.848571,0.0
88333,4307,2020_21_KC_TB,2021020700,TB,KC,POST,21,TB,KC,TB,...,88,VICTORY FORMATION,8.0,82,46,26,21,0,59.848571,0.0
88334,4328,2020_21_KC_TB,2021020700,TB,KC,POST,21,TB,KC,TB,...,88,VICTORY FORMATION,8.0,82,46,26,21,0,59.848571,0.0


In [ ]:
# Temperature Grading using Percentiles
temperature_percentiles = np.percentile(pbp['Temperature'], [25, 50, 75])

def grade_temperature(temp_value):
    if temp_value <= temperature_percentiles[0]:  # Below 25th percentile (low)
        return 'Low'
    elif temp_value <= temperature_percentiles[1]:  # Between 25th and 50th percentile (moderate)
        return 'Moderate'
    elif temp_value <= temperature_percentiles[2]:  # Between 50th and 75th percentile (high)
        return 'High'
    else:  # Above 75th percentile (very high)
        return 'Very High'

pbp['temperature_grade'] = pbp['Temperature'].apply(grade_temperature)

# Creating goal line situation flag for short-yardage plays
pbp['goal_line_situation'] = ((pbp['yardline_100'] <= 10) & (pbp['YardsToGo'] <= 3)).astype(int)

# Season Progression
pbp['season_progression'] = pbp['week'] / 17.0  

# Late Season Fatigue
pbp['late_season'] = (pbp['week'] > 12).astype(int)

pbp['is_close_game'] = ((abs(pbp['score_differential']) <= 7) & (pbp['game_seconds_remaining'] < 900)).astype(int)

# Define temperature buckets
def categorize_temperature(temp):
    if temp < 32:
        return 'Extreme Cold'
    elif temp > 90:
        return 'Extreme Heat'
    else:
        return 'Moderate'

# Apply function to create temperature categories
pbp['temperature_bucket'] = pbp['Temperature'].apply(categorize_temperature)

# Define bad weather flag
pbp['is_bad_weather'] = ((pbp['Precipitation'].notnull()) | 
                         (pbp['Temperature'] < 32) | 
                         (pbp['Temperature'] > 90)).astype(int)

# Defining field condition based on precipitation
pbp['poor_field_condition'] = ((pbp['surface_type'] == 'grass') & (pbp['Precipitation'].notnull())).astype(int)

# Calculate rest differential
pbp['rest_differential'] = pbp['HOME_day_since_last_game'] - pbp['AWAY_day_since_last_game']

# Identify if a team is on short rest
pbp['short_rest'] = (pbp[['HOME_day_since_last_game', 'AWAY_day_since_last_game']].min(axis=1) < 7).astype(int)

# Calculate pass-run ratio
pbp['pass_run_ratio'] = pbp['pass_count'] / (pbp['pass_count'] + pbp['run_count'])

# Offensive predictability 
pbp['offensive_predictability'] = abs(pbp['pass_run_ratio'] - 0.5)

# Classifying 3rd and long situations
pbp['third_and_long'] = ((pbp['Down'] == 3) & (pbp['YardsToGo'] > 7)).astype(int)

# Classifying fourth down plays
pbp['fourth_down_attempt'] = ((pbp['Down'] == 4) & (pbp['play_type'] != 'punt') & (pbp['play_type'] != 'field_goal')).astype(int)

# Blitz Situation
pbp['blitz_situation'] = (pbp['defenders_in_box'] > 6).astype(int)

In [5]:
pbp.fillna(0,inplace=True)

In [6]:
pbp.to_csv('../../Data/play-by-play/pbp_exp10.csv', index=False)